### Callin Switzer

1 Nov 2016

-Read in all the Kalmia polygon images (from R KalmiaVisualizePolygons_V2.R)

-Weight pixels with increasing values as they get closer to zero

-Resample all of the images

-Stack resampled images

-Visualize a heatmap

-Save heatmap

In [1]:
import cv2

import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

In [2]:
%qtconsole

In [3]:
# list files in directory

In [4]:
# load colormap
os.chdir("/Users/cswitzer/Dropbox/ExperSummer2016/Kalmia/Manuscript/DatasetsSupplemental/KalmiaDigitizedPolygon/colormap")
import matplotlib.cm as cm
import colormaps as cmaps
colormap_r = matplotlib.colors.ListedColormap(cmaps.magma.colors[::-1])

In [5]:



os.chdir("/Users/cswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaExamples/KalmiaManualTrig_VidsToProcess")

In [6]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
photoList = [ x for x in onlyfiles if not x.startswith('.') and x.endswith('.png')]

In [7]:
len(photoList)

29

In [8]:
# visualize individual polygons with weighting

kk = 0
fpth = photoList[kk]
img = cv2.imread(fpth,0)
img = (img < 255) * 1.0

# these are the points of interest
intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])

#img[:, 1000]= 0.9
#img[1694, :] = 0.9
plt.clf()
io.imshow(img, cmap = 'gray')
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])



# make multiplication matrix
mask = np.ones((2000, 2000))

# make points exponentially decreasing 
# this makes the center probability  == 10 times the outside probability
row1 = (10.0**((np.hstack([range(0, 1000), range(999, -1, -1)]))/231.0))

# normalize
row1 = row1 / np.max(row1)

# make a matrix to multiply by
mask2 = mask * (row1 )
io.imshow(mask2 * img, cmap = 'gray')
print row1[768], row1[1000], row1[1231]
print  row1[1000] / float(row1[768])

0.1 1.0 0.1
10.0


In [25]:
# stack all polygons and visualize without weighting
stackedImg = np.ones((2000, 2000)) * 0.0



for ii in range(len(photoList)):
    # read in image
    
    fpth = photoList[ii]
    img = cv2.imread(fpth,0)
    img = (img < 255) * 1.0
    if ii == 0:
        img = cv2.flip(img, 1)
    img2 = mask * img
    stackedImg = img2 + stackedImg

plt.clf()
io.imshow(stackedImg, cmap = 'YlOrRd')

In [26]:
# convert 0's to NAN
stackedImg = stackedImg / np.max(stackedImg)
stackedImg[stackedImg == 0.0] = np.nan

In [27]:
# remove border
a = stackedImg
slice_val = 150
b = a[slice_val:-slice_val, slice_val:-slice_val]

In [30]:
plt.close("all")


levels =  np.arange(0, 1.2, 0.1)
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
#ax.tick_params('off')
CF = ax.imshow(b, cmap = colormap_r)
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
#cbar.ax.set_yticklabels(["0", "2", "3", "4"])
plt.tight_layout()

fig.savefig("/Users/cswitzer/Desktop/RawStacked_29Polygons.png", dpi = 700)



In [31]:
# resample stacked images without weighting
def resampleImgsNoWeight():
    # stack all polygons and visualize
    stackedImg = np.ones((2000, 2000)) * 0.0

    # make multiplication matrix
    mask = np.ones((2000, 2000))

    photoSamp = np.random.choice(range(len(photoList)), size=len(photoList), replace=True, p=None)


    for ii in photoSamp:
        # read in imag
        fpth = photoList[ii]
        img = cv2.imread(fpth,0)
        img = (img < 255) * 1.0



        #flip the one that I did backwards
        if fpth == '20160615_121016.png':
            img = cv2.flip(img, 1)

        img2 = mask * img

        # stack images
        stackedImg = img2 + stackedImg
    
    return stackedImg

In [34]:
img1 = resampleImgsNoWeight()

In [35]:
plt.clf()
io.imshow(img1, cmap = colormap_r)

In [36]:
stt = time.time()
imStt = np.array([resampleImgsNoWeight() for ii in range(100)])
print time.time() - stt

296.615514994


In [55]:
# sum up arrays to make total bootstrap resample 
imSum = imStt.sum(axis = 0)
i2 = imSum

# normalize
imSum = imSum / np.max(imSum)

plt.clf()
io.imshow(imSum, cmap = colormap_r)
plt.xlim([600, 1400])
plt.ylim([ 1700, 700])
plt.scatter(x = intPoints[:,0], y = intPoints[:,1])

In [48]:
imSum[imSum == 0] = np.nan

In [49]:
levels =  np.arange(0, 1.2, 0.1)
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
#ax.tick_params('off')
CF = ax.imshow(imSum, cmap = colormap_r)
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])
plt.tight_layout()

fig.savefig("/Users/cswitzer/Desktop/RawStacked_29Polygons_withPoints.png", dpi = 700)

In [50]:
a = imSum
slice_val = 150
c = a[slice_val:-slice_val, slice_val:-slice_val]

In [51]:
levels =  np.arange(0, 1.2, 0.1)
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
#ax.tick_params('off')
CF = ax.imshow(c, cmap = colormap_r)
cbar = fig.colorbar(CF, ticks = levels, fraction = 0.031, pad = 0)
plt.tight_layout()

fig.savefig("/Users/cswitzer/Desktop/RawStacked_29Polygons_NoPoints.png", dpi = 700)

In [52]:
os.path.abspath(photoList[kk])

'/Users/cswitzer/Dropbox/ExperSummer2016/Kalmia/KalmiaExamples/KalmiaManualTrig_VidsToProcess/20160615_121016.png'

In [83]:
# add  blur
plt.clf()
kernel = np.ones((50,50),np.float32)/25
imSum2 = np.array(cv2.filter2D(i2.copy(),-1,kernel))

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')


imNA = imSum2
imNA[imNA <= 10] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r)
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')

# add contour lines
ax.contour(imSum2, levels = range(10, 100, 20), colors = 'k')
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('SingleAntherHeatmap_magmaCmap_blurr_withPoints.png', dpi = 500)

In [85]:
# add  blur
plt.clf()
kernel = np.ones((50,50),np.float32)/25
imSum2 = np.array(cv2.filter2D(i2.copy(),-1,kernel))

# normalize
imSum2 = imSum2 / np.max(imSum2) * 100


plt.close('all')


imNA = imSum2
imNA[imNA <= 10] = np.nan
fig, ax = plt.subplots()
im = ax.imshow(imNA, cmap = colormap_r)
cax = fig.add_axes([0.78, 0.2, 0.05, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical')

# add contour lines
ax.contour(imSum2, levels = range(10, 100, 20), colors = 'k')
# plt.xlim([600, 1400])
# plt.ylim([ 1700, 700])

ax.axis('off')

ax.set_xlim([600, 1900])
ax.set_ylim([ 1800, 700])

intPoints = np.array([[768, 1694],[1000, 1694],[1231, 1694]])
#ax.scatter(x = intPoints[:,0], y = intPoints[:,1])

plt.show()

plt.savefig('SingleAntherHeatmap_magmaCmap_blurr_NoPoints.png', dpi = 500)